In [1]:
### calculate daily average air temperature from hourly average and place onto EASE 2.0 grid

In [2]:
import numpy as np
from scipy.stats import pearsonr
import datetime
import pandas as pd
import glob
import matplotlib.pyplot as plt
from matplotlib import gridspec
import datetime
from regrid import regrid
import xarray as xr
import seaborn as sns
import os
import tqdm
from netCDF4 import Dataset
import warnings
warnings.filterwarnings("ignore")

/opt/anaconda3/lib/python3.8/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [37]:
# creating numpy are with daily temperature data on EASE grid for 2010-2020

datapath = '/Users/carmennab/Dropbox/OI_PolarSnow/data/ERA5/temp/'
EASE_lats = np.load('/Users/carmennab/Dropbox/OI_PolarSnow/EASE/auxiliary/new_lat_25km.npy')
EASE_lons = np.load('/Users/carmennab/Dropbox/OI_PolarSnow/EASE/auxiliary/new_lon_25km.npy')
missing = np.full((153,360,360),np.nan)
years = ['2010', '2011', '2012','2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

all_temps = {}

for year in tqdm.tqdm(years):
    
    # load ERA5 temperature data
    file = datapath + year +'.nc'
    data = Dataset(file,'r')
    lon = data['longitude']
    lat = data['latitude']
    era5_lon,era5_lat = np.meshgrid(np.array(lon),np.array(lat))
    temp = data['t2m']    
           
    # turn hourly data into daily data and regrid onto EASE grid        
    temp_daily = []

    for i in np.arange(0,temp.shape[0],24):

        temps_on_day = temp[i:i+24]

        day_ave = np.nanmean(temps_on_day,axis=0)
    

        regrid_ave = regrid(day_ave,
                            era5_lon,
                            era5_lat,
                            EASE_lons,
                            EASE_lats,
                            method='linear')

        temp_daily.append(regrid_ave)
    
    # create full year list, with NaNs for days in May - September
    year_temps = np.concatenate((temp_daily[:-92],missing,temp_daily[-92:]),axis=0)

    all_temps[year] = year_temps

100%|████████████████████████████████████████| 11/11 [1:29:38<00:00, 488.99s/it]


In [38]:
# create one array containing data for all years and save
all_temps_arr = np.concatenate((all_temps['2010'], all_temps['2011'], all_temps['2012'], all_temps['2013'], 
                               all_temps['2014'], all_temps['2015'], all_temps['2016'], all_temps['2017'], 
                               all_temps['2018'], all_temps['2019'], all_temps['2020']),axis=0)

In [39]:
# create temp NETCDF
days = np.arange(0,4018)
data_vars = {'Temperature':(['t','x','y'],all_temps_arr)}
ds = xr.Dataset(data_vars = data_vars,
                coords={'Longitude':(['x','y'],EASE_lons),
                        'Latitude':(['x','y'],EASE_lats),
                         'Day':(['t'],days)}     )
ds.to_netcdf(f'/Users/carmennab/Dropbox/OI_PolarSnow/EASE/data/ERA5_temp.nc','w')